In [1]:
import pandas as pd

In [2]:
import pandas as pd

# Определение типов данных для столбцов
dtypes = {
    'uid': 'object',
    'polis_number': 'object',
    'date_z1': 'object',  # Оставляем как object для последующего преобразования
    'sh_code': 'object',
    'sh_name': 'object',
    'lpu_short_name': 'object',
    'lpu_subject': 'object',
    'karina_line_row': 'object',
    'karina_etalon': 'float64',
    'line_karina': 'int64',
    'karina_support_scheme': 'object',
    'drugs_in_scheme': 'object',
    'start_therapy': 'object',
    'last_cycle_end': 'object',
    'diagnosis_date': 'object',
    'age_at_diagnosis': 'float64',
    'diagnosis': 'object',
    'morphology_type_icdo3': 'object',
    'prr_stage': 'object',
    'prr_last_contact': 'object',
    'prr_death_date': 'object',
    'registry_line': 'object',
    'registry_line2': 'object',
    'drug_line': 'object',
    'cr_match': 'object',
    'cr_match_corrected': 'object',
    'last_contact_date_reestr': 'object',
    'stage_reestr': 'object',
    'death_date_reestr': 'object',
    'cause_of_death': 'object',
    'last_diagnosis_reestr': 'object',
    'tariff': 'object'
}

# Чтение CSV
df = pd.read_csv(
    r'C:\GitHub\analytics\data_c56.csv',
    sep=';',
    dtype=dtypes,
    na_values=['#Н/Д', ''],  # Обработка специальных значений как NA
    on_bad_lines='skip'
)

# Преобразование столбцов с датами в datetime
date_columns = ['date_z1', 'start_therapy', 'last_cycle_end', 'diagnosis_date',
                'prr_last_contact', 'prr_death_date', 'last_contact_date_reestr', 'death_date_reestr']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%d.%m.%Y', errors='coerce')

# Преобразование tariff в float64, замена запятой на точку
df['tariff'] = df['tariff'].str.replace(',', '.').astype('float64')

# Преобразование karina_etalon и age_at_diagnosis в int64 после обработки NA
df['karina_etalon'] = df['karina_etalon'].fillna(0).astype('int64')
df['age_at_diagnosis'] = df['age_at_diagnosis'].fillna(0).astype('int64')

# Вывод первых 5 строк и информации о типах данных
df.head()
df.dtypes

uid                                 object
polis_number                        object
date_z1                     datetime64[ns]
sh_code                             object
sh_name                             object
lpu_short_name                      object
lpu_subject                         object
karina_line_row                     object
karina_etalon                        int64
line_karina                          int64
karina_support_scheme               object
drugs_in_scheme                     object
start_therapy               datetime64[ns]
last_cycle_end              datetime64[ns]
diagnosis_date              datetime64[ns]
age_at_diagnosis                     int64
diagnosis                           object
morphology_type_icdo3               object
prr_stage                           object
prr_last_contact            datetime64[ns]
prr_death_date              datetime64[ns]
registry_line                       object
registry_line2                      object
drug_line  

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51451 entries, 0 to 51450
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   uid                       51451 non-null  object        
 1   polis_number              51451 non-null  object        
 2   date_z1                   51451 non-null  datetime64[ns]
 3   sh_code                   51451 non-null  object        
 4   sh_name                   51451 non-null  object        
 5   lpu_short_name            51451 non-null  object        
 6   lpu_subject               51451 non-null  object        
 7   karina_line_row           51451 non-null  object        
 8   karina_etalon             51451 non-null  int64         
 9   line_karina               51451 non-null  int64         
 10  karina_support_scheme     51451 non-null  object        
 11  drugs_in_scheme           51451 non-null  object        
 12  start_therapy     

In [4]:
df.head()

,uid,polis_number,date_z1,sh_code,sh_name,lpu_short_name,lpu_subject,karina_line_row,karina_etalon,line_karina,...,registry_line2,drug_line,cr_match,cr_match_corrected,last_contact_date_reestr,stage_reestr,death_date_reestr,cause_of_death,last_diagnosis_reestr,tariff
0,0149620890000059_43930,149620890000059,2020-04-09,sh675,Паклитаксел 175-200 мг/м? в 1-й день + цисплат...,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. ПЕТРОВА"" МИНЗДРА...",г. Санкт-Петербург,01.01.,1,1,...,Последующие циклы линии (кроме последнего),Паклитаксел + цисплатин01,соот КР,соот КР,2024-06-18,IIIC,NaT,0,C56,56007.21
1,0149620890000059_43972,149620890000059,2020-05-21,sh672,Паклитаксел 175-225 мг/м? в 1-й день + карбопл...,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. ПЕТРОВА"" МИНЗДРА...",г. Санкт-Петербург,01.01.,1,1,...,Последующие циклы линии (кроме последнего),Карбоплатин + паклитаксел01,соот КР,соот КР,2024-06-18,IIIC,NaT,0,C56,61173.19
2,0149620890000059_44001,149620890000059,2020-06-19,sh672,Паклитаксел 175-225 мг/м? в 1-й день + карбопл...,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. ПЕТРОВА"" МИНЗДРА...",г. Санкт-Петербург,01.01.,1,1,...,Последующие циклы линии (кроме последнего),Карбоплатин + паклитаксел01,соот КР,соот КР,2024-06-18,IIIC,NaT,0,C56,61173.19
3,0149620890000059_44022,149620890000059,2020-07-10,sh672,Паклитаксел 175-225 мг/м? в 1-й день + карбопл...,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. ПЕТРОВА"" МИНЗДРА...",г. Санкт-Петербург,01.01.,1,1,...,Последующие циклы линии (кроме последнего),Карбоплатин + паклитаксел01,соот КР,соот КР,2024-06-18,IIIC,NaT,0,C56,61173.19
4,0149620890000059_44047,149620890000059,2020-08-04,sh672,Паклитаксел 175-225 мг/м? в 1-й день + карбопл...,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. ПЕТРОВА"" МИНЗДРА...",г. Санкт-Петербург,01.01.,1,1,...,Последующие циклы линии (кроме последнего),Карбоплатин + паклитаксел01,соот КР,соот КР,2024-06-18,IIIC,NaT,0,C56,61173.19
